In [1]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

dirs = ["..", "/home/hanlv/workspace/code/research/infodemic/LLM/autogen/create_prompt_bing_llm"]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

import prompt_rag

from swift.llm import (
    get_model_tokenizer, get_template, inference, ModelType, get_default_template_type
)
from swift.tuners import Swift
from custom import CustomModelType, CustomTemplateType

model_dir = '/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm/output/openchat_3.5/with_solar_info/brave/data1-split=8:2-ratio=1.0/lr=1.1e-4-20240116-00:52:51/checkpoint-609'
model_type = CustomModelType.openchat_35
template_type = CustomTemplateType.openchat_35

model, tokenizer = get_model_tokenizer(model_type, model_kwargs={'device_map': 'auto'})

model = Swift.from_pretrained(model, model_dir, inference_mode=True)
model.generation_config.max_new_tokens = 512
template = get_template(template_type, tokenizer)


2024-01-19 11:57:50,253 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-01-19 11:57:50,255 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-01-19 11:57:50,282 - modelscope - INFO - Loading done! Current index file version is 1.11.0, with md5 dc3cda2b56a86b0242cd145399a14071 and a total number of 953 components indexed
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MistralConfig {
  "_name_or_path": "/home/css/models/openchat-3.5-0106",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": fa

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/hanlv/miniconda3/envs/swift/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [6]:
x = {"query": "According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.\n\nCLAIM:\nPublication date: 2020-08-13\nContent: Illinois Strengthens Face Mask Rules in Businesses\n\nPRIOR KNOWLEDGE:\nInformation 1:\nPublication date: 2013-08-06\nTitle: Illinois Strengthens Face Mask Rules in Businesses - WebMD\nContent:\nCOVID-19 is a new type of coronavirus that causes mild to severe cases. Here’s a quick guide on how to spot symptoms, risk factors, prevent spread of the disease, and find out what to do if you think you have it.\nInformation 2:\nPublication date: None\nTitle: press-release\nContent:\nIllinoisans can resume activities without wearing a mask indoors on February 28th except where required by federal, state, local, tribal, or territorial laws, rules, and regulations, including local business and workplace guidance. Federal requirements, in effect through at least March 18, include all transportation systems such as airports, planes, trains, and buses. \"Preparing to repeal statewide masking mandates at the end of the month is aggressive and optimistic but reasonable,\" said Dr.\n\"Broad mandates are not about individuals. They are put in place to help protect communities, businesses, and healthcare access. Repealing the mask mandate allows people to choose the mitigation layers that are best for them and I have no doubt that many should and will choose to keep mask rules.\"\nIf these trends continue — and we expect them to —then on Monday, February 28th, we will lift the indoor mask requirement for the State of Illinois,\" said Governor JB Pritzker. \"I want to be clear: Many local jurisdictions, businesses and organizations have their own mask requirements and other mitigations that must be respected.\nSeth Trueger, a Northwestern emergency physician who is also immunocompromised. \"Masking has helped slow the spread even in the face of omicron's transmissibility. We can and must use this time to further increase vaccination uptake & outreach, especially among children and other populations with low vaccination rates, so when the next wave comes, we will be even better prepared.\"\nInformation 3:\nPublication date: None\nTitle: Mask and Vaccine Requirements FAQ's\nContent:\nOn September 3, 2021, the Governor signed Executive Order 21-22 which requires all individuals over the age of 2 and who can medically tolerate a face covering to wear a face covering when in indoor public places. The Executive Order also requires health care workers, school personnel, higher education personnel and students, and employees and contractors of state-owned or operated congregate facilities to be fully vaccinated, as described in the Order\nInformation 4:\nPublication date: None\nTitle: FAQ for Businesses Concerning Use of Face-Coverings During COVID-19\nContent:\nA: Businesses reserve the right to refuse service to persons unable to comply with the requirement to wear a face covering, but they are required to provide a reasonable accommodation if it does not cause an undue hardship. Businesses are encouraged to inform customers there are exceptions to the requirement that all individuals must wear a mask.\nThese frequently asked questions are to provide guidance regarding the application of the face covering requirement in Executive Order 2021-10 for businesses and other places of public accommodation subject to Article 5 of the Illinois Human Rights Act, 775 ILCS 5/. A: A face covering is a mask or cloth face covering that covers your nose and mouth.\nExceptions may be made for individuals with medical conditions or disabilities that prevent them from safely wearing a face covering. For more information, refer to the questions on reasonable accommodations. A: Masks still must be worn by everyone on planes, buses, trains, and other forms of public transportation; in transportation hubs, such as airports and train and bus stations; in health care settings; and in congregate facilities, such as correctional facilities and homeless shelters.\nThese frequently asked questions are to provide guidance regarding the application of the face covering requirement in Executive Order 2021-10 for businesses and other places of public accommodation subject to Article 5 of the Illinois Human Rights Act, 775 ILCS 5/. When Face Coverings are Required Q: What does it mean to wear a face covering?\nInformation 5:\nPublication date: 2020-04-22\nTitle: Facing Your Face Mask Duties – A List of Statewide Orders | Littler ...\nContent:\n********************************************************NOTE: Given the reduction in activity on this topic,THIS POST WILL NO LONGER BE UPDATED, as of November 10, 2022.********************************************************Governors and public health officials across the country implemented stringent mitigation measures to help contain the spread of COVID-19.\n\nCLAIM: Publication date: 2020-08-13, Content: Illinois Strengthens Face Mask Rules in Businesses\n\nTo accurately classify this claim, we need to analyze the relevant information provided. \n\nInformation 1: This is from 2013 and is not directly related to the claim about Illinois strengthening face mask rules in businesses in 2020. It provides general information about COVID-19 symptoms, risk factors, and prevention.\n\nInformation 2: This information confirms that Illinois lifted its indoor mask requirement on February 28, 2022, except where required by specific laws, rules, and regulations. It also mentions that the state was optimistic about this decision due to the decreasing trends of COVID-19 cases. The information does not directly contradict the claim about strengthening face mask rules in 2020, but it provides context about the situation in 2021-2022.\n\nInformation 3: This information reveals that on September 3, 2021, the Governor of Illinois signed an Executive Order requiring face coverings for individuals over the age of 2 in indoor public places. It also mandated vaccination for specific groups. This information supports the claim that Illinois strengthened face mask rules in businesses in 2020, as the Executive Order was issued in September 2021, which falls within the timeframe of the claim.\n\nInformation 4: This information provides guidance for businesses regarding the use of face coverings during the COVID-19 pandemic. It explains the definition of a face covering, exceptions for medical conditions or disabilities, and situations where masks are still required. This information does not directly contradict the claim but provides additional context about the implementation of face mask rules in businesses.\n\nInformation 5: This information is a list of statewide orders regarding face masks, but it is important to note that it was last updated on November 10, 2022, and the claim is about 2020. However, it does highlight the efforts of governors and public health officials to contain the spread of COVID-19 through stringent mitigation measures.\n\nIn conclusion, based on the available information, particularly Information 3, which confirms the implementation of stricter face mask rules in businesses in Illinois in September 2021, we can reasonably infer that the claim about Illinois strengthening face mask rules in businesses in 2020 is TRUE. However, it is essential to note that the specific action mentioned in the claim might not have occurred in 2020 but rather in 2021.", "response": "TRUE."}


In [7]:
query = x["query"]

response, history = inference(
    model, template, query, 
    verbose=False, # 带有聊天模板
)    
response
# print(f'query: {query}')
# print(f'response: {response}')


# # 流式输出对话模板
# inference(model, template, '第一个问题是什么', history, verbose=True, stream=True)


'TRUE.'